In [6]:
using JLD2, FileIO
stats = load("../stats.jld2")

Dict{String, Any} with 7 entries:
  "rl90"   => Dict{Tuple{String, String}, Vector{T} where T}(("mod-5", "3DS")=>…
  "rl9"    => Dict{Tuple{String, String}, Vector{T} where T}(("mod-5", "3DS")=>…
  "rl360"  => Dict{Tuple{String, String}, Vector{T} where T}(("mod-5", "3DS")=>…
  "ls30"   => Dict{Tuple{String, String}, Vector{T} where T}(("mod-5", "3DS")=>…
  "roam"   => Dict{Tuple{String, String}, Vector{T} where T}(("mod-5", "3DS")=>…
  "ls"     => Dict{Tuple{String, String}, Vector{T} where T}(("mod-5", "3DS")=>…
  "roam90" => Dict{Tuple{String, String}, Vector{T} where T}(("mod-5", "3DS")=>…

In [7]:
statnames = sort!(collect(keys(stats)))
conditions = sort!(collect(reduce(∪, keys(d) for d in values(stats))))

6-element Vector{Tuple{String, String}}:
 ("N2", "4DS")
 ("N2", "NS")
 ("mod-5", "3DS")
 ("mod-5", "NS")
 ("tph-1", "4DS")
 ("tph-1", "NS")

In [8]:
using Statistics, Unzip
using StatsBase
using Plots

#stage_sp(cond, well, stage) = cond2well2traj[cond][well].speed[stage_frames(well, stage; stagedict)]
#stage_roam(well_i, stage) = roam_for_stage(exwells[well_i]..., trajs[well_i], stage)

function bins(ax, nbins)
    edges = round.(Int, range(first(ax), last(ax)+1, length=nbins+1))
    [edges[i]:edges[i+1]-1 for i in 1:nbins]
end
fin(x) = !ismissing(x) && isfinite(x)
bin_means(v::AbstractVector, nbins) = [mean(view(v,i)) for i in bins(eachindex(v), nbins)]
bin_mean_fin(v::AbstractVector, nbins) = [mean(filter(fin,view(v,i))) for i in bins(eachindex(v), nbins)]

function bin_means_weights(v, nbins)
    b = bins(eachindex(v), nbins)
    f = [filter(fin, view(v,i)) for i in b]
    means = mean.(f)
    weights = length.(f) ./ length.(b)
    (; means, weights)
end

using PaddedViews

function pad!(rows, val, α=1)
    len = maximum(length.(rows))
    for (i,row) in pairs(rows)
        d = len - length(row)
        nfront = round(Int, d*α)
        nback = d - nfront
        
        rows[i] = PaddedView(val, row, (1:len,), (nfront+1:len-nback,))
    end
    rows
end

pad! (generic function with 2 methods)

In [21]:
using Interact
@manipulate throttle=0.1 for conds = togglebuttons(conditions, multiple=true, value=[("N2", "NS"), ("N2", "4DS")]), 
                stat in statnames,
                stage = 1:5, nbins=50:5:150, 
                pad=true, α in slider(0:0.01:1, value=0),
                crange = rangepicker(0:0.01:1)
    sort!(conds)
    stat_rows = reduce(vcat, 
                        sort(stats[stat][cond][stage]; by=length) for cond in conds; init=[])
    #sort!(stat_rows; by=length)
    if isempty(stat_rows)
        plot()
    else
        pad && pad!(stat_rows, missing, α)
        means, weights = unzip(bin_means_weights.(stat_rows, nbins))
        M = reduce(hcat, means)'
        Mmin, Mmax = extrema(filter(isfinite,M))
        cmin, cmax = Mmin .+ crange .* (Mmax-Mmin)
        heatmap(M, legend=false, cbar=true, clim=(cmin,cmax))
        cond_ends = cumsum(length(stats[stat][cond][stage]) for cond in conds)
        hline!(cond_ends .+ 0.5, c="white")
        yticks!((midpoints([0;cond_ends]), join.(conds, " ")))
        #size(M)
    end
end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :span), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any[], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "text : key")))], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "click: function (){\$root.update(val)}, css: {'is-primary is-selected' : \$root.index().includes(val), 'interact-widget is-medium button ' : true}")))], Dict{Symbol, Any}(:attributes => ("data-bind" => "foreach : options_js"), :className => "buttons has-addons is-centered")), Dict{String, Tuple{Observables.AbstractObservable, Union{Nothing, Bool}}}("options_js" => (Observable{Any} with 1 listeners. Value:
OrderedDict[OrderedDict{String, Any}("key" => "(\"N2\", \"4DS\")", "val" => 1, "id" => "idLmRiDTeD"), OrderedDict{String, Any}("key" => "(\"N2\", \"NS\")", "val" => 2, "id" => "idjOfmMjoq"), OrderedDict{String, Any}("key" => "(\"mod-5\", \"3DS\")", "val" => 3, "id" => "idjaL6hCne"), OrderedDict{String, Any}("key" => "(\"mod-5\", \"NS\")", "val" => 4, "id" => "idhD1s9XVz"), OrderedDict{String, Any}("key" => "(\"tph-1\", \"4DS\")", "val" => 5, "id" => "idivxU299P"), OrderedDict{String, Any}("key" => "(\"tph-1\", \"NS\")", "val" => 6, "id" => "idvIFZfjZH")], nothing), "index" => (Observable{Any} with 2 listeners. Value:
[2, 1], nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/home/sternlab/.julia/packages/Knockout/IP1uR/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/home/sternlab/.julia/packages/Knockout/IP1uR/src/../assets/knockout_punches.js"), Asset("js", nothing, "/home/sternlab/.julia/packages/InteractBase/Zfu5P/src/../assets/all.js"), Asset("css", nothing, "/home/sternlab/.julia/packages/InteractBase/Zfu5P/src/../assets/style.css"), Asset("css", nothing, "/home/sternlab/.julia/packages/Interact/SbgIk/src/../assets/bulma_confined.min.css")], Dict{Any, Any}("options_js" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"options_js\"]()) ? (this.valueFromJulia[\"options_js\"]=true, this.model[\"options_js\"](val)) : undefined})")], "index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Condition(Base.InvasiveLinkedList{Task}(Task (runnable) @0x00007f407b6c82b0, Task (runnable) @0x00007f407b6c82b0), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"options_js\":WebIO.getval({\"name\":\"options_js\",\"scope\":\"13861863696644001740\",\"id\":\"13451799945821078258\",\"type\":\"observable\"}),\"index\":WebIO.getval({\"name\":\"index\",\"scope\":\"13861863696644001740\",\"id\":\"13091146978405444121\",\"type\":\"o

In [14]:
n2m(a) = replace(a, NaN=>missing)
finites(x) = filter(fin,x)

function spstats(stat; nbins=250)
    # multiplication with strong 0: 0⋆NaN==0
    a ⊛ b = iszero(a) || iszero(b) ? zero(promote_type(typeof(a),typeof(b))) : a*b

    t = midpoints(range(0,1,length=nbins+1))
    means, weights = unzip(bin_means_weights.(stat, nbins))
    bin_means = mapreduce(n2m, hcat, means)'
    μ = mean.(filter.(fin, eachcol(bin_means)))
    weights = reduce(hcat, weights)'
    
    # TODO right way to deal with missing values?
    μ_c = coalesce.(bin_means,NaN)
    μ_d = (μ_c .- μ') .⊛ weights ./ .√(sum(weights; dims=1).-1)
    Σ = μ_d' * μ_d

    μ, Σ, μ_d, bin_means, weights
end

@manipulate for stat in statnames,
                condition in conditions,
                stage in 1:5,
                nbins = 1:500,
                zero_mean = false
    μ, Σ, μ_d, bin_means, weights = spstats(stats[stat][condition][stage]; nbins)

    zero_mean && (bin_means .-= μ')
    q_low, med, q_high = unzip([quantile(filter(fin,c), (0.25,0.5,0.75)) for c in eachcol(bin_means)])
    d_plt = plot(med, ribbon = (med.-q_low, q_high.-med) , label="quartiles")
    plot!([mean(finites(c)) for c in eachcol(bin_means)], label="mean")
    plot!(reduce(hcat, quantile(filter(fin,c), [0.05,0.95]) for c in eachcol(bin_means))', 
                label=["5%" "95%"], c="black", alpha=0.2)
    plot(heatmap(bin_means), d_plt, layout=(2,1), size=(600,600))
end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["stat"], Dict{Symbol, Any}(:className => "interact ", :style => Dict{Any, Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol, Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :span), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any[], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "text : key")))], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "click: function (){\$root.update(val)}, css: {'is-primary is-selected' : \$root.index() == val, 'interact-widget is-medium button ' : true}")))], Dict{Symbol, Any}(:attributes => ("data-bind" => "foreach : options_js"), :className => "buttons has-addons is-centered"))], Dict{Symbol, Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[], Dict{Symbol, Any}())], Dict{Symbol, Any}(:className => "interact-flex-row-right"))], Dict{Symbol, Any}(:className => "interact-flex-row interact-widget")), Dict{String, Tuple{Observables.AbstractObservable, Union{Nothing, Bool}}}("options_js" => (Observable{Any} with 1 listeners. Value:
OrderedDict[OrderedDict{String, Any}("key" => "ls", "val" => 1, "id" => "idaYGYECQa"), OrderedDict{String, Any}("key" => "ls30", "val" => 2, "id" => "idCkgp5fHV"), OrderedDict{String, Any}("key" => "rl360", "val" => 3, "id" => "idhgxqdvi1"), OrderedDict{String, Any}("key" => "rl9", "val" => 4, "id" => "idLxjXQDvF"), OrderedDict{String, Any}("key" => "rl90", "val" => 5, "id" => "idmHM55D08"), OrderedDict{String, Any}("key" => "roam", "val" => 6, "id" => "idWYtcbfQT"), OrderedDict{String, Any}("key" => "roam90", "val" => 7, "id" => "idyjjLEGil")], nothing), "index" => (Observable{Any} with 2 listeners. Value:
1, nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/home/sternlab/.julia/packages/Knockout/IP1uR/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/home/sternlab/.julia/packages/Knockout/IP1uR/src/../assets/knockout_punches.js"), Asset("js", nothing, "/home/sternlab/.julia/packages/InteractBase/Zfu5P/src/../assets/all.js"), Asset("css", nothing, "/home/sternlab/.julia/packages/InteractBase/Zfu5P/src/../assets/style.css"), Asset("css", nothing, "/home/sternlab/.julia/packages/Interact/SbgIk/src/../assets/bulma_confined.min.css")], Dict{Any, Any}("options_js" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"options_js\"]()) ? (this.valueFromJulia[\"options_js\"]=true, this.model[\"options_js\"](val)) : undefined})")], "index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Condition(Base.InvasiveLinkedList{Task}(Task (runnable) @0x00007f40b564dba0, Task (runnable) @0x00007f40b564dba0), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\

In [15]:
using Distances
using StatsBase
using StatsBase: normalize

#cum_dist(h1, h2) = mean(abs(x-y) for (x,y) in zip(cumsum(normalize(h1,1)), cumsum(normalize(h2,1))))
#jsnorm(h1, h2) = js_divergence(normalize(h1,1), normalize(h2,1))
js_bits(x,y) = js_divergence(x,y) / log(2)
hellinger²(x,y) = hellinger(x,y)^2

n_time_bins = 150
n_stat_bins = 50

@manipulate for d in [hellinger, hellinger², js_bits], stat in statnames, 
                condition in conditions, stage in 1:5,
                n_time_bins = 1:300,
                n_stat_bins = 1:200

    μ, Σ, μ_d, bin_means, weights = spstats(stats[stat][condition][stage]; nbins=n_time_bins)
    #s = 0:0.025:1
    s = range(extrema(finites(bin_means))...; length=n_stat_bins+1)
    #s = -1.5:0.01:1

    @assert minimum(s) <= minimum(finites(bin_means))
    @assert maximum(s) >= maximum(finites(bin_means))

    W = reduce(hcat, normalize(fit(Histogram, finites(c), s), mode=:probability).weights for c in eachcol(bin_means))
    C = [d(c1, c2) for c1 in eachcol(W), c2 in eachcol(W)]

    a = 100
    b = size(bin_means,2) - a*minimum(finites(bin_means))
    function binned_mean_inset_plot!()
        q_low, med, q_high = unzip([quantile(filter(fin,c), (0.25,0.5,0.75)) for c in eachcol(bin_means)])
        plot!(b.+a.*med, ribbon = ((med.-q_low) .* a, (q_high.-med) .* a) , label="quartiles")
        plot!(b.+a.*μ, label="mean", c=1)
        plot!(reduce(hcat, b.+a.*quantile(finites(c), [0.05,0.95]) for c in eachcol(bin_means))', 
                    label=["5%" "95%"], c="black", alpha=0.2)
    end

    dist_d_hm = heatmap(C, aspect_ratio=1, clims=(0,1), 
            title="$(join(condition," ")) (n=$(size(bin_means,1))) stage $stage\n $stat, $d ($n_stat_bins bins)",
            #legend=:outerbottom,
            legend=false,
            )
    binned_mean_inset_plot!()

    dist_d_hist = histogram(vec(C), legend=false, yaxis=false, yticks=false, xlims=(0,1))

    S = corspearman(μ_d)
    cor_hm = heatmap(S, c=:diverging, 
                    title="spearman",
                    legend=false,
                    #legend=:outerbottom
                    clims=(-1,1), aspect_ratio=1, 
                    )
    binned_mean_inset_plot!()

    cor_hist = histogram(vec(S), legend=false, yaxis=false, yticks=false, xlims=(-1,1))
    vline!([0])

    plot(dist_d_hm, cor_hm, dist_d_hist, cor_hist,
        layout=grid(2,2, heights=(0.8,0.2)),
        size=(600,600))

    #savefig("$outdir/$stat-$stage-$n_time_bins-$d-$n_stat_bins-$(join(cond,"-")).png"); current()
end


Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["d"], Dict{Symbol, Any}(:className => "interact ", :style => Dict{Any, Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol, Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :span), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any[], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "text : key")))], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "click: function (){\$root.update(val)}, css: {'is-primary is-selected' : \$root.index() == val, 'interact-widget is-medium button ' : true}")))], Dict{Symbol, Any}(:attributes => ("data-bind" => "foreach : options_js"), :className => "buttons has-addons is-centered"))], Dict{Symbol, Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[], Dict{Symbol, Any}())], Dict{Symbol, Any}(:className => "interact-flex-row-right"))], Dict{Symbol, Any}(:className => "interact-flex-row interact-widget")), Dict{String, Tuple{Observables.AbstractObservable, Union{Nothing, Bool}}}("options_js" => (Observable{Any} with 1 listeners. Value:
OrderedDict[OrderedDict{String, Any}("key" => "hellinger", "val" => 1, "id" => "idHtTc3yxF"), OrderedDict{String, Any}("key" => "hellinger²", "val" => 2, "id" => "idvIPxiol3"), OrderedDict{String, Any}("key" => "js_bits", "val" => 3, "id" => "id1uwb9Cua")], nothing), "index" => (Observable{Any} with 2 listeners. Value:
1, nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/home/sternlab/.julia/packages/Knockout/IP1uR/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/home/sternlab/.julia/packages/Knockout/IP1uR/src/../assets/knockout_punches.js"), Asset("js", nothing, "/home/sternlab/.julia/packages/InteractBase/Zfu5P/src/../assets/all.js"), Asset("css", nothing, "/home/sternlab/.julia/packages/InteractBase/Zfu5P/src/../assets/style.css"), Asset("css", nothing, "/home/sternlab/.julia/packages/Interact/SbgIk/src/../assets/bulma_confined.min.css")], Dict{Any, Any}("options_js" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"options_js\"]()) ? (this.valueFromJulia[\"options_js\"]=true, this.model[\"options_js\"](val)) : undefined})")], "index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Condition(Base.InvasiveLinkedList{Task}(Task (runnable) @0x00007f406edc4940, Task (runnable) @0x00007f406edc4940), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n     

In [12]:
using MultivariateStats

isfullrow(r) = all(!ismissing,r)
fullrows(a::AbstractMatrix{T}) where T = mapreduce(Vector{nonmissingtype(T)}, hcat, 
                            r for r in eachrow(a) if isfullrow(r))'

@manipulate for stat in statnames, condition in conditions, stage in 1:5, nbins=1:300,
                pc = slider(1:20, value=1)
    μ, Σ, μ_d, mean_mat, weights = spstats(stats[stat][condition][stage]; nbins)

    mean_mat_f = fullrows(mean_mat)

    pca_t = fit(PCA, mean_mat_f')
    Pt = projection(pca_t)

    m_plt = plot(mean(pca_t), lw=3, label="mean")
    pca_plt = plot()
    v = principalvars(pca_t)[pc]
    plot!(m_plt, mean(pca_t), ribbon = √v*Pt[:,pc], label="", c=2, la=0, fa=0.2)
    plot!(m_plt, mean(pca_t) .+ √v .* Pt[:,pc], label="±σ_$pc⋅PC_$pc", c=2, lw=1)
    plot!(pca_plt, Pt[:,pc], fillrange=0, label="", c=2)
    plot(m_plt, pca_plt, layout=(2,1))
end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["stat"], Dict{Symbol, Any}(:className => "interact ", :style => Dict{Any, Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol, Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :span), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any[], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "text : key")))], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "click: function (){\$root.update(val)}, css: {'is-primary is-selected' : \$root.index() == val, 'interact-widget is-medium button ' : true}")))], Dict{Symbol, Any}(:attributes => ("data-bind" => "foreach : options_js"), :className => "buttons has-addons is-centered"))], Dict{Symbol, Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[], Dict{Symbol, Any}())], Dict{Symbol, Any}(:className => "interact-flex-row-right"))], Dict{Symbol, Any}(:className => "interact-flex-row interact-widget")), Dict{String, Tuple{Observables.AbstractObservable, Union{Nothing, Bool}}}("options_js" => (Observable{Any} with 1 listeners. Value:
OrderedDict[OrderedDict{String, Any}("key" => "ls", "val" => 1, "id" => "idyeFxKIhQ"), OrderedDict{String, Any}("key" => "ls30", "val" => 2, "id" => "idvOKyKhOz"), OrderedDict{String, Any}("key" => "rl360", "val" => 3, "id" => "idipUG05sN"), OrderedDict{String, Any}("key" => "rl9", "val" => 4, "id" => "idj5iQbTWC"), OrderedDict{String, Any}("key" => "rl90", "val" => 5, "id" => "idQbBHZKlx"), OrderedDict{String, Any}("key" => "roam", "val" => 6, "id" => "idkhTonfvW"), OrderedDict{String, Any}("key" => "roam90", "val" => 7, "id" => "idwUNLSRNi")], nothing), "index" => (Observable{Any} with 2 listeners. Value:
1, nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/home/sternlab/.julia/packages/Knockout/IP1uR/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/home/sternlab/.julia/packages/Knockout/IP1uR/src/../assets/knockout_punches.js"), Asset("js", nothing, "/home/sternlab/.julia/packages/InteractBase/Zfu5P/src/../assets/all.js"), Asset("css", nothing, "/home/sternlab/.julia/packages/InteractBase/Zfu5P/src/../assets/style.css"), Asset("css", nothing, "/home/sternlab/.julia/packages/Interact/SbgIk/src/../assets/bulma_confined.min.css")], Dict{Any, Any}("options_js" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"options_js\"]()) ? (this.valueFromJulia[\"options_js\"]=true, this.model[\"options_js\"](val)) : undefined})")], "index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Condition(Base.InvasiveLinkedList{Task}(Task (runnable) @0x00007f406ecf3c70, Task (runnable) @0x00007f406ecf3c70), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\

In [18]:
isfullrow(r) = all(!ismissing,r)

fullrows(a::AbstractMatrix{T}) where T = mapreduce(Vector{nonmissingtype(T)}, hcat, 
                            r for r in eachrow(a) if isfullrow(r))'

@manipulate for stat in statnames, condition in conditions, stage in 1:5, nbins=1:300,
                by = togglebuttons(["worms", "time bins"]), show_mean = false, n_pcs = slider(1:20, value=4, label="PCs")
    μ, Σ, μ_d, mean_mat, weights = spstats(stats[stat][condition][stage]; nbins)
    bycols = by == "time bins"

    mean_mat_f = fullrows(mean_mat)
    x = bycols ? mean_mat_f : mean_mat_f'
    pca = fit(PCA, x)
    μ, P = mean(pca), projection(pca)
    P .*= sign(sum(P[:,1]))
    pcs = P[:,1:n_pcs]
    pc_labels = ["PC $i" for i=1:n_pcs]
    coeff_labels = ["c $i" for i=1:n_pcs]
    coeffs = (x .- μ)' * pcs
    col_vecs   = bycols ? show_mean ? hcat(μ, pcs) : pcs : coeffs
    col_labels = bycols ? show_mean ? vcat(["μ"], pc_labels) : pc_labels : coeff_labels
    # mean is at the end so the colors of PCAs are not affected and match between subplots
    row_vecs   = bycols ? coeffs : show_mean ? hcat(pcs, μ) : pcs
    ncol, nrow = size(col_vecs,2), size(row_vecs,2)

    pvars_plt = bar(principalvars(pca)[1:n_pcs] ./ tvar(pca), legend=false, 
                     group = 1:n_pcs)
    plot!(cumsum(principalvars(pca)[1:n_pcs] ./ tvar(pca)), marker=true)
    plot(
        plot(heatmap(mean_mat_f, title=condition),
            #bar(mean(pca), orientation=:horizontal, axis=false, ticks=false, title="μ"),
            #(bar(P[:,i], orientation=:horizontal, axis=false, ticks=false, title="PC$i") for i=1:n_pcs)...,
            (bar(v, orientation=:horizontal, axis=false, ticks=false; title) 
                    for (title,v) in zip(col_labels, eachcol(col_vecs))
            )...,
            legend=false, link=:y,
            layout=grid(1,ncol+1,widths=normalize([10;fill(1,ncol)],1))),
        plot(
            #plot(mean_mat_f' * P[:,1:n_pcs], label=(1:n_pcs)', legend=false), 
            plot(row_vecs, label=(1:n_pcs)', legend=false), 
            pvars_plt, 
            layout=(1,2)),
        layout=(2,1), size=(600,600))
end


Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["stat"], Dict{Symbol, Any}(:className => "interact ", :style => Dict{Any, Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol, Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :span), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any[], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "text : key")))], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "click: function (){\$root.update(val)}, css: {'is-primary is-selected' : \$root.index() == val, 'interact-widget is-medium button ' : true}")))], Dict{Symbol, Any}(:attributes => ("data-bind" => "foreach : options_js"), :className => "buttons has-addons is-centered"))], Dict{Symbol, Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[], Dict{Symbol, Any}())], Dict{Symbol, Any}(:className => "interact-flex-row-right"))], Dict{Symbol, Any}(:className => "interact-flex-row interact-widget")), Dict{String, Tuple{Observables.AbstractObservable, Union{Nothing, Bool}}}("options_js" => (Observable{Any} with 1 listeners. Value:
OrderedDict[OrderedDict{String, Any}("key" => "ls", "val" => 1, "id" => "id3U1hUNbK"), OrderedDict{String, Any}("key" => "ls30", "val" => 2, "id" => "idcKGJKqH3"), OrderedDict{String, Any}("key" => "rl360", "val" => 3, "id" => "idW0v3oXuT"), OrderedDict{String, Any}("key" => "rl9", "val" => 4, "id" => "iddUF8E5HF"), OrderedDict{String, Any}("key" => "rl90", "val" => 5, "id" => "id7t90l1qX"), OrderedDict{String, Any}("key" => "roam", "val" => 6, "id" => "idrMAEdgU4"), OrderedDict{String, Any}("key" => "roam90", "val" => 7, "id" => "idT6o8cP7f")], nothing), "index" => (Observable{Any} with 2 listeners. Value:
1, nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/home/sternlab/.julia/packages/Knockout/IP1uR/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/home/sternlab/.julia/packages/Knockout/IP1uR/src/../assets/knockout_punches.js"), Asset("js", nothing, "/home/sternlab/.julia/packages/InteractBase/Zfu5P/src/../assets/all.js"), Asset("css", nothing, "/home/sternlab/.julia/packages/InteractBase/Zfu5P/src/../assets/style.css"), Asset("css", nothing, "/home/sternlab/.julia/packages/Interact/SbgIk/src/../assets/bulma_confined.min.css")], Dict{Any, Any}("options_js" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"options_js\"]()) ? (this.valueFromJulia[\"options_js\"]=true, this.model[\"options_js\"](val)) : undefined})")], "index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Condition(Base.InvasiveLinkedList{Task}(Task (runnable) @0x00007f406edc53c0, Task (runnable) @0x00007f406edc53c0), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\